In [1]:
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy import sparse

In [6]:
SEED = 777

In [7]:
torch.manual_seed(SEED)

#만약 GPU가 사용가능한 환경이라면 GPU를 사용
if torch.cuda.is_available():
    use_cuda = True

device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [105]:
import os
import pandas as pd
from scipy import sparse
import numpy as np
from typing import Tuple

def get_count(tp: pd.DataFrame, id) -> pd.Series:
    playcount_groupbyid = tp[[id]].groupby(id, as_index=True)
    count = playcount_groupbyid.size()

    return count

# 특정한 횟수 이상의 리뷰가 존재하는(사용자의 경우 min_uc 이상, 아이템의 경우 min_sc이상) 
# 데이터만을 추출할 때 사용하는 함수입니다.
# 현재 데이터셋에서는 결과적으로 원본그대로 사용하게 됩니다.
def filter_triplets(tp, min_uc=5, min_sc=0) -> Tuple[pd.DataFrame, pd.Series, pd.Series]:
    if min_sc > 0:
        itemcount = get_count(tp, 'item')
        tp = tp[tp['item'].isin(itemcount.index[itemcount >= min_sc])]

    if min_uc > 0:
        usercount = get_count(tp, 'user')
        tp = tp[tp['user'].isin(usercount.index[usercount >= min_uc])]

    usercount, itemcount = get_count(tp, 'user'), get_count(tp, 'item')
    return tp, usercount, itemcount

In [101]:
print("Load and Preprocess Movielens dataset")
# Load Data
DATA_DIR = '/opt/ml/input/data/train'
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'train_ratings.csv'), header=0)

# Filter Data
raw_data, user_activity, item_popularity = filter_triplets(raw_data, min_uc=5, min_sc=0)

Load and Preprocess Movielens dataset


In [104]:
print("Shuffle User Indices")
unique_uid = user_activity.index
print("(BEFORE) unique_uid:",unique_uid.values)
np.random.seed(SEED)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]
print("(AFTER) unique_uid:",unique_uid.values)

n_users = unique_uid.size #31360
n_test_users = 3000


# Split Train/Validation/Test User Indices
tr_users = unique_uid[:(n_users - n_test_users)]
te_users = unique_uid[(n_users - n_test_users):]

#주의: 데이터의 수가 아닌 사용자의 수입니다!
print("훈련 데이터에 사용될 사용자 수:", len(tr_users))
print("테스트 데이터에 사용될 사용자 수:", len(te_users))

Shuffle User Indices
(BEFORE) unique_uid: [    11     14     18 ... 138486 138492 138493]
(AFTER) unique_uid: [ 55113 117964  27674 ...  70054  66411 136047]
훈련 데이터에 사용될 사용자 수: 28360
테스트 데이터에 사용될 사용자 수: 3000


In [12]:
train_plays = raw_data.loc[raw_data['user'].isin(tr_users)]

In [97]:
train_plays.shape

(4668600, 3)

In [14]:
test_plays = raw_data.loc[raw_data['user'].isin(te_users)]

In [96]:
test_plays.shape

(485871, 3)

In [18]:
test_plays['user'].unique()[0]

65

In [85]:
test_plays[test_plays['user']==65].head()

,user,item,time
1682,65,1235,1303761145
1683,65,3107,1303761159
1684,65,3555,1303761188
1685,65,2431,1303761224
1686,65,3910,1303761398


In [89]:
for usr_id, tp in test_plays.groupby('user', as_index=False):
    usr_id, tp['item']
    if not isinstance(usr_id, int) and not isinstance(tp, pd.DataFrame):
        print('except')
        break